In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from multi_gene_inference import multi_data_loader, MultiGCNInferenceNetwork, multi_train, validate

In [2]:
HOME = '/home/jupyter-dylan/'    # Dylan
devices = torch.device('cuda')

## Load data

In [3]:
# load gene names
#gene_name = np.loadtxt('gene_name.txt', dtype=str)     # Jiayu
gene_name = np.concatenate(                            # Dylan
    [np.loadtxt('{}gene labels/input_genes.txt'.format(HOME), dtype=str),
    np.loadtxt('{}gene labels/output_genes-1.txt'.format(HOME), dtype=str)]
)

# gene_edge = np.loadtxt('genemania_edge.txt', dtype=str)   # Jiayu
with open('../genemania_edges.csv', newline='') as f:         # Dylan
    reader = csv.reader(f)
    gene_edge = list(reader)

In [4]:
# create mapping table from genes to nodes number
node_map = {}
for i, j in enumerate(gene_name):
    node_map.update({j:i})

In [5]:
gene_set = set(["App", "Apoe", "Gusb","Lamp5","Mbp","Pvalb","S100b","Slc30a3","Snca","Mapt"])

In [6]:
# load training data
#train_loader = data_loader('input_train_cat.csv', gene='gusb', node_map=node_map,    # Jiayu
#                           edge_list=edge_list, multiplier=1e-5)
train_loader = multi_data_loader('{}new_data/input_train.csv'.format(HOME), genes=gene_set, node_map=node_map,    # Dylan
                           gene_edge=gene_edge, cat='{}new_data/output_train-1.csv'.format(HOME))

# load validation data
#validate_loader = data_loader('input_test_cat.csv', gene='gusb', node_map=node_map,  # Jiayu
#                              edge_list=edge_list, multiplier=1e-5)
validate_loader = multi_data_loader('{}new_data/input_test.csv'.format(HOME), genes=gene_set, node_map=node_map,  # Dylan
                              gene_edge=gene_edge, cat='{}new_data/output_test-1.csv'.format(HOME))

In [7]:
print(len(train_loader))
print(len(validate_loader))

185420
46360


In [8]:
print(train_loader[0].x.size())
print(train_loader[0].y.size())
print(train_loader[0].edge_index.size())

torch.Size([1431, 1])
torch.Size([1, 1])
torch.Size([2, 488342])


## Train GCN

In [ ]:
model = MultiGCNInferenceNetwork().to(torch.device('cuda'))
train_loss_lis = []
validate_loss_lis = []
train_batch_size = 64
validate_batch_size = 64
epochs = 100

for i in range(epochs): 
    
    train_loss = multi_train(model, train_loader, train_batch_size)
    train_loss_lis.append(train_loss)
    
    val_loss = validate(model, validate_loader, validate_batch_size)
    validate_loss_lis.append(val_loss)
    
    # print ('epochs: %d || train_losses: %f || train_mse: %f || val_loss: %f || val_mse: %f' \
          # %(i, train_loss, train_mse, val_loss, val_mse))
    # print ('epochs: %d || train_losses: %f' %(i, train_loss))

  9%|▉         | 275/2897 [03:24<32:22,  1.35it/s, loss=0.101]

In [ ]:
from gene_inference import MultiGCNInferenceNetwork

In [ ]:
# ep = [i for i in range(20)]
plt.plot(range(len(validate_loss_lis)), validate_loss_lis)
plt.plot(range(len(train_loss_lis)), train_loss_lis)